# Quy trình Tiền xử lý Dữ liệu (Data Preprocessing Pipeline)

Notebook này thực hiện quy trình làm sạch dữ liệu thô từ các file văn bản (.txt) được tổ chức theo thư mục chủ đề.

**Các bước xử lý chính:**
1.  **Làm sạch nội dung (Cleaning):** Quét toàn bộ file, loại bỏ các dòng chứa đường dẫn (URL) hoặc metadata rác.
2.  **Tải và Chuẩn hóa:** Đọc dữ liệu, xử lý các lỗi mã hóa (Encoding) phổ biến (UTF-8, UTF-16).
3.  **Khử trùng lặp (Deduplication):** Loại bỏ các bài viết có nội dung giống nhau hoàn toàn.
4.  **Lọc bài viết ngắn (Short Content Filter):** Loại bỏ các bài viết không đủ độ dài tối thiểu (ví dụ: < 100 ký tự).
5.  **Lọc nhiễu bằng ML (Label Cleaning):** Sử dụng mô hình Logistic Regression để phát hiện và loại bỏ các bài viết bị đặt sai thư mục chủ đề.
6.  **Xuất dữ liệu:** Lưu bộ dữ liệu sạch ra file `nlp_dataset.jsonl` để sử dụng cho huấn luyện.

In [ ]:
# 1. THIẾT LẬP MÔI TRƯỜNG
import os
import shutil
import hashlib
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import LabelEncoder
from pyvi import ViTokenizer

# Cấu hình đường dẫn
CURRENT_DIR = Path.cwd()
PROJECT_ROOT = CURRENT_DIR if (CURRENT_DIR / "data").exists() else CURRENT_DIR.parent

RAW_DATA_DIR = PROJECT_ROOT / "data" / "raw"      # Nơi chứa các folder .txt gốc
FINAL_DATA_DIR = PROJECT_ROOT / "data" / "final"  # Nơi lưu jsonl sạch
DISCARD_DIR = PROJECT_ROOT / "data" / "discarded" # Nơi chứa các file bị loại bỏ

# Tạo các thư mục cần thiết
for d in [FINAL_DATA_DIR, DISCARD_DIR]:
    d.mkdir(parents=True, exist_ok=True)
    
print(f"Thư mục dữ liệu gốc: {RAW_DATA_DIR}")
print(f"Thư mục lưu kết quả: {FINAL_DATA_DIR}")
print(f"Thư mục chứa file loại bỏ: {DISCARD_DIR}")

# 2. LÀM SẠCH NỘI DUNG (URL REMOVAL)
Bước này quét qua các file `.txt`, loại bỏ các dòng chứa liên kết web (http/https) vì chúng không đóng góp vào việc phân loại văn bản.

In [ ]:
def clean_file_content(file_path):
    """Đọc file, loại bỏ dòng chứa URL và ghi lại file."""
    try:
        # Thử đọc với UTF-16 trước (do dữ liệu gốc hay dùng format này)
        try:
            with open(file_path, 'r', encoding='utf-16') as f:
                lines = f.readlines()
        except UnicodeError:
            with open(file_path, 'r', encoding='utf-8') as f:
                lines = f.readlines()
        
        # Lọc bỏ dòng chứa URL
        clean_lines = [line for line in lines if "http" not in line]
        
        # Ghi đè lại file (luôn lưu UTF-8 cho thống nhất)
        if len(clean_lines) != len(lines):
            with open(file_path, 'w', encoding='utf-8') as f:
                f.writelines(clean_lines)
            return True # Có thay đổi
    except Exception as e:
        print(f"Lỗi xử lý file {file_path.name}: {e}")
    return False

print("Đang quét và làm sạch URL trong các file văn bản...")
files = list(RAW_DATA_DIR.glob("**/*.txt"))
changed_count = 0

for file_path in tqdm(files, desc="URL Cleaning"):
    if clean_file_content(file_path):
        changed_count += 1

print(f"Đã xử lý xong. Số file được làm sạch: {changed_count}/{len(files)}")

# 3. TẢI DỮ LIỆU VÀO DATAFRAME
Đọc toàn bộ file `.txt` vào Pandas DataFrame để dễ dàng thao tác lọc và xử lý.

In [ ]:
def load_data_to_df(data_dir):
    data = []
    files = list(data_dir.glob("**/*.txt"))
    
    print(f"Đang tải {len(files)} file vào bộ nhớ...")
    for file_path in tqdm(files, desc="Loading Data"):
        content = ""
        try:
            # Ưu tiên đọc UTF-16, fallback sang UTF-8
            try:
                with open(file_path, "r", encoding="utf-16") as f: 
                    content = f.read().strip()
            except:
                with open(file_path, "r", encoding="utf-8") as f: 
                    content = f.read().strip()
            
            if content:
                # Chuẩn hóa Unicode ngay lập tức
                content = unicodedata.normalize('NFC', content)
                data.append({
                    "raw_text": content,
                    "label_name": file_path.parent.name,
                    "filename": file_path.name,
                    "file_path": str(file_path)
                })
        except Exception as e:
            print(f"Không thể đọc file {file_path}: {e}")
            continue
            
    return pd.DataFrame(data)

df = load_data_to_df(RAW_DATA_DIR)
print(f"Đã tải thành công: {len(df)} dòng dữ liệu.")
display(df.head())

# 4. LỌC TRÙNG LẶP VÀ BÀI VIẾT QUÁ NGẮN
- **Deduplication:** Sử dụng hàm băm (MD5) để phát hiện bài trùng.
- **Short Text Filter:** Loại bỏ bài viết dưới 70 ký tự (thường là lỗi crawl hoặc tiêu đề cụt).

In [ ]:
def move_discarded_files(df_discard, reason_folder):
    """Di chuyển các file bị loại bỏ vào thư mục riêng để kiểm tra lại nếu cần."""
    target_dir = DISCARD_DIR / reason_folder
    target_dir.mkdir(parents=True, exist_ok=True)
    
    for _, row in df_discard.iterrows():
        src = Path(row['file_path'])
        dst = target_dir / f"{row['label_name']}_{row['filename']}"
        try:
            if src.exists():
                shutil.move(str(src), str(dst))
        except Exception as e:
            print(f"Lỗi di chuyển file {src.name}: {e}")

# 1. Xử lý trùng lặp
df['content_hash'] = df['raw_text'].apply(lambda x: hashlib.md5(x.encode()).hexdigest())
duplicates = df[df.duplicated(subset='content_hash', keep='first')]
df_clean = df.drop_duplicates(subset='content_hash', keep='first')

print(f"Phát hiện {len(duplicates)} bài viết trùng lặp. Đang di chuyển...")
move_discarded_files(duplicates, "duplicates")

# 2. Xử lý bài viết ngắn
MIN_LENGTH = 100
short_articles = df_clean[df_clean['raw_text'].str.len() < MIN_LENGTH]
df_clean = df_clean[df_clean['raw_text'].str.len() >= MIN_LENGTH]

print(f"Phát hiện {len(short_articles)} bài viết quá ngắn (<{MIN_LENGTH} ký tự). Đang di chuyển...")
move_discarded_files(short_articles, "too_short")

print(f"\nDữ liệu còn lại sau lọc sơ bộ: {len(df_clean)}")

# 5. LỌC NHIỄU NHÃN BẰNG MACHINE LEARNING (LABEL CLEANING)
Sử dụng mô hình Logistic Regression để tìm các bài viết nằm sai thư mục (Outliers/Mislabeling).
Ví dụ: Bài viết về "Thể thao" nhưng lại nằm trong thư mục "Kinh doanh".

In [ ]:
print("Đang chuẩn bị dữ liệu để lọc nhiễu nhãn...")

# Tiền xử lý sơ bộ (Tách từ) cho mô hình lọc nhiễu
tqdm.pandas(desc="Tokenizing for Cleaning")
df_clean['text_tokenized'] = df_clean['raw_text'].progress_apply(ViTokenizer.tokenize)

# Vector hóa TF-IDF
tfidf_cleaner = TfidfVectorizer(max_features=10000)
X = tfidf_cleaner.fit_transform(df_clean['text_tokenized'])

# Mã hóa nhãn
le_cleaner = LabelEncoder()
y = le_cleaner.fit_transform(df_clean['label_name'])

# Huấn luyện Logistic Regression và dự đoán (Cross Validation để tránh Overfitting)
print("Đang chạy Cross-Validation để phát hiện nhãn sai (có thể mất vài phút)...")
clf = LogisticRegression(max_iter=1000, n_jobs=-1)

# Lấy xác suất dự đoán cho từng mẫu
y_probs = cross_val_predict(clf, X, y, cv=3, method='predict_proba')
y_pred = y_probs.argmax(axis=1)
y_conf = y_probs.max(axis=1)

# Xác định các mẫu nhiễu
# Tiêu chí: Mô hình dự đoán khác nhãn gốc VÀ độ tin cậy > 0.85 (Rất chắc chắn là sai)
CONFIDENCE_THRESHOLD = 0.85
mask_wrong = (y_pred != y) & (y_conf > CONFIDENCE_THRESHOLD)
wrong_indices = df_clean.index[mask_wrong]

df_wrong = df_clean.loc[wrong_indices].copy()
df_wrong['predicted_label'] = le_cleaner.inverse_transform(y_pred[mask_wrong])

print(f"\nPhát hiện {len(df_wrong)} bài viết có khả năng bị gán nhãn sai (Độ tin cậy > {CONFIDENCE_THRESHOLD}).")
if len(df_wrong) > 0:
    print("Ví dụ 3 mẫu sai:")
    display(df_wrong[['label_name', 'predicted_label', 'raw_text']].head(3))

# Loại bỏ các mẫu sai khỏi tập dữ liệu chính
df_final = df_clean.drop(index=wrong_indices)
move_discarded_files(df_wrong, "wrong_label_detected")

print(f"\nKích thước dữ liệu cuối cùng: {len(df_final)}")

# 6. XUẤT DỮ LIỆU SẠCH (FINAL EXPORT)
Lưu dữ liệu đã qua xử lý ra file `nlp_dataset.jsonl`. Đây sẽ là đầu vào cho file `EDA.ipynb` và `model.ipynb`.

In [ ]:
# Tạo cột 'text' chính thức (đã tách từ) để dùng cho các bước sau
df_final['text'] = df_final['text_tokenized']

# Chỉ giữ lại các cột cần thiết
cols_to_save = ['text', 'raw_text', 'label_name', 'filename']
df_export = df_final[cols_to_save]

print(f"Đang lưu dữ liệu ra: {JSONL_PATH}")
df_export.to_json(JSONL_PATH, orient='records', lines=True, force_ascii=False)

print("✅ Quy trình tiền xử lý hoàn tất!")
print("Sẵn sàng chuyển sang bước EDA và Modeling.")

# Thống kê cuối cùng theo chủ đề
print("\nThống kê số lượng bài viết theo chủ đề:")
print(df_export['label_name'].value_counts())